In [1]:
import numpy as np
import pandas as pd

import tensorflow as tf

In [2]:
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("MNIST_data/", one_hot = True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [92]:
#Optimizers

learning_rate = 0.003
epochs = 21
batch_size = 100

#Training data placeholders

x = tf.placeholder(tf.float32, [None, 784])

#Output data placeholder - 10 digits

y = tf.placeholder(tf.float32, [None, 10])

In [93]:
#Declare weights for connectors from i/p to hl1

W1 = tf.Variable(tf.random_normal([784, 300], stddev = 0.03), name = 'W1')
b1 = tf.Variable(tf.random_normal([300], stddev = 0.03), name = "b1")

#Connectors from hl1 to o/p

W2 = tf.Variable(tf.random_normal([300, 10], stddev = 0.03), name = 'W2')
b2 = tf.Variable(tf.random_normal([10], stddev = 0.03, name = 'b2'))

In [94]:
#Calculate the output of the hidden layer

hidden_out = tf.add(tf.matmul(x, W1), b1)       #W1*x + b1
hidden_out = tf.nn.relu(hidden_out)             #relu activation

#dropout activations
#tf.nn.dropout(pkeep) #probability of 50-75% = pkeep

In [95]:
#Hidden layer output - softmax output layer

y_ = tf.nn.softmax(tf.add(tf.matmul(hidden_out,W2), b2))

In [96]:
#to keep the y_ value clipped in (0,1)

y_clipped = tf.clip_by_value(y_, 1e-10, 0.9999999)

#Cost function to optimize

cross_entropy = -tf.reduce_mean(tf.reduce_sum(y * tf.log(y_clipped) + (1 - y)*tf.log(1 - y_clipped), axis = 1))

In [97]:
#Optimizer

optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(cross_entropy)

In [98]:
#Initialisation

init_op = tf.global_variables_initializer()

#Define accuracy

correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [99]:
#Training

with tf.Session() as sess:
    sess.run(init_op)
    total_batch = int(len(mnist.train.labels)/ batch_size)
    writer = tf.summary.FileWriter("output", sess.graph)
    for epoch in range(epochs):
        avg_cost = 0
        for i in range(total_batch):
            batch_x, batch_y = mnist.train.next_batch(batch_size = batch_size)
            _, c = sess.run([optimizer, cross_entropy],
                           feed_dict = {x: batch_x, y: batch_y})
            avg_cost += c/ total_batch
        print("Epoch:", (epoch + 1), "cost =", "{:.3f}".format(avg_cost))
        if (epoch%5 == 0):
            print(sess.run(accuracy*100, feed_dict = {x: mnist.test.images, y: mnist.test.labels}),"%")
    writer.close()

Epoch: 1 cost = 0.417
96.18 %
Epoch: 2 cost = 0.179
Epoch: 3 cost = 0.119
Epoch: 4 cost = 0.088
Epoch: 5 cost = 0.066
Epoch: 6 cost = 0.055
97.7 %
Epoch: 7 cost = 0.049
Epoch: 8 cost = 0.040
Epoch: 9 cost = 0.039
Epoch: 10 cost = 0.031
Epoch: 11 cost = 0.027
97.96 %
Epoch: 12 cost = 0.029
Epoch: 13 cost = 0.029
Epoch: 14 cost = 0.022
Epoch: 15 cost = 0.029
Epoch: 16 cost = 0.022
98.2 %
Epoch: 17 cost = 0.014
Epoch: 18 cost = 0.023
Epoch: 19 cost = 0.017
Epoch: 20 cost = 0.034
Epoch: 21 cost = 0.018
97.86 %
